# 09: Cells and cell methods

Sometimes a data value does not represent a point, but instead some coordinate range that we can call a *cell*. Some examples could be

* Maximum temperatures within each month
* Values in latitude/longitude bins
* Mean sea ice salinity from melting a chunk of an ice core between two depth values

In this tutorial we will look at some examples of how you can encode the limits of each cell in CF-NetCDF.

## Maximum monthly temperatures

Let's start with the basic code to create our xarray object for a time series of temperature values.

Units of *months since date* or *years since date* are not recommended in CF because of potential for confusion with leap years. Therefore, we can use days since 1970-01-01 (the epoch) for first day of each month for 2023.

In [1]:
import datetime
import xarray as xr
import calendar
import numpy as np

days_since_epoch_month_start = []
year = 2023
epoch = datetime.date(1970, 1, 1)

for month in range(1, 13):
    first_day_of_month = datetime.date(year, month, 1)
    time_difference = (first_day_of_month - epoch).days
    days_since_epoch_month_start.append(time_difference)

print(days_since_epoch_month_start)


[19358, 19389, 19417, 19448, 19478, 19509, 19539, 19570, 19601, 19631, 19662, 19692]


In [2]:
maximum_monthly_temperatures = [4.6,5.2,7.1,12.3,17.8,21.3,24.6,22.8,19.0,14.2,8.8,6.1]

xrds = xr.Dataset(
    coords={
        'time': days_since_epoch_month_start
    },
    data_vars={
        'Temperature': ('time', maximum_monthly_temperatures)
    } 
)

xrds['Temperature'].attrs = {
    'standard_name': 'air_temperature',
    'long_name': 'Maximum air temperatures per month',
    'units': 'Degrees Celsius',
    'coverage_content_type': 'physicalMeasurement'
}
xrds['time'].attrs = {
    'standard_name': 'time',
    'long_name': 'time in months',
    'units': 'days since 1970-01-01',
    'coverage_content_type': 'coordinate'
}
xrds

<xarray.Dataset>
Dimensions:      (time: 12)
Coordinates:
  * time         (time) int64 19358 19389 19417 19448 ... 19631 19662 19692
Data variables:
    Temperature  (time) float64 4.6 5.2 7.1 12.3 17.8 ... 22.8 19.0 14.2 8.8 6.1

But our values are not representative of just one day; they are maximum values for the month! How can we let the user know is in an unambiguous, machine-readable way? Remember - the long name is free text, so we can't rely on this.

We need to include some bounds to define where each month begins and end. Let's now calculate the number of days since the epoch for the last day of each month.

In [3]:
days_since_epoch_month_end = []

for month in range(1, 13):
    # Find the last day of the month
    last_day_of_month = calendar.monthrange(year, month)[1]
    last_day_date = datetime.date(year, month, last_day_of_month)
    
    # Calculate the days since epoch for the last day of the month
    time_difference = (last_day_date - epoch).days
    days_since_epoch_month_end.append(time_difference)

print(days_since_epoch_month_end)

[19388, 19416, 19447, 19477, 19508, 19538, 19569, 19600, 19630, 19661, 19691, 19722]


Let's now combine these into a 2D array for the bounds of each month

In [4]:
time_bounds_2d_array = np.array(list(zip(days_since_epoch_month_start, days_since_epoch_month_end)))
print(time_bounds_2d_array)

[[19358 19388]
 [19389 19416]
 [19417 19447]
 [19448 19477]
 [19478 19508]
 [19509 19538]
 [19539 19569]
 [19570 19600]
 [19601 19630]
 [19631 19661]
 [19662 19691]
 [19692 19722]]


Now we need to create a new variable in our xarray object for the time bounds. This variable needs to have 2 dimensions. *time* of course, but also another dimension that we will call *nv* for number of vertices. This dimension will have a value of 2 in this case because the cell provides information on only the minimum and maximum time.

In [5]:
xrds = xrds.expand_dims(nv=2) # Creating new dimension

xrds['time_bounds'] = (['time','nv'], time_bounds_2d_array)
xrds

<xarray.Dataset>
Dimensions:      (nv: 2, time: 12)
Coordinates:
  * time         (time) int64 19358 19389 19417 19448 ... 19631 19662 19692
Dimensions without coordinates: nv
Data variables:
    Temperature  (nv, time) float64 4.6 5.2 7.1 12.3 17.8 ... 19.0 14.2 8.8 6.1
    time_bounds  (time, nv) int64 19358 19388 19389 19416 ... 19691 19692 19722

Now we need to add metadata to make this machine readable. Below we are saying that the *time_bounds* variable defines the bounds of the *time* variable. We are using *cell_methods* to state that values are the maximums within each cell with respect to time.

In [6]:
xrds['time'].attrs['bounds'] = 'time_bounds'
xrds['Temperature'].attrs['cell_methods'] = 'time: maximum'

A full list of possible cell methods that you can use is provided here: https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#appendix-cell-methods

This includes `maximum`, `minimum`, `mean`, `median`,`standard_deviation` and more.

## Ice core data

Here is a full example for ice core data. You might have a pandas dataframe something like what we are creating below.

In [7]:
import pandas as pd

# Sample data
data = {
    'minimum_depth': [0, 10, 20, 30],
    'maximum_depth': [10, 20, 30, 38],
    'salinity': [35.4, 35.6, 35.5, 35.2]
}

# Creating the DataFrame
df = pd.DataFrame(data)
df

,minimum_depth,maximum_depth,salinity
0,0,10,35.4
1,10,20,35.6
2,20,30,35.5
3,30,38,35.2


Here is the code to get this into an xarray object with bounds

In [8]:
xrds = xr.Dataset(
    coords={
        'depth': df['minimum_depth'] # Could be any value within the cell
    },
    data_vars={
        'Salinity': ('depth', df['salinity'])
    } 
)

xrds = xrds.expand_dims(nv=2)

depths_2d_array = df[['minimum_depth','maximum_depth']].to_numpy()

xrds['depth_bounds'] = (['depth','nv'], depths_2d_array)

xrds['Salinity'].attrs = {
    'standard_name': 'sea_ice_salinity',
    'long_name': 'Salinity of sea ice measured by melting chunks of an ice core',
    'units': '1e-3',
    'coverage_content_type': 'physicalMeasurement',
    'cell_methods': 'depth: mean'
}
xrds['depth'].attrs = {
    'standard_name': 'depth',
    'long_name': 'depth in ice core',
    'units': 'cm',
    'positive': 'down',
    'coverage_content_type': 'coordinate',
    'bounds': 'depth_bounds'
}
xrds



<xarray.Dataset>
Dimensions:       (nv: 2, depth: 4)
Coordinates:
  * depth         (depth) int64 0 10 20 30
Dimensions without coordinates: nv
Data variables:
    Salinity      (nv, depth) float64 35.4 35.6 35.5 35.2 35.4 35.6 35.5 35.2
    depth_bounds  (depth, nv) int64 0 10 10 20 20 30 30 38